## Push model to hub

In [1]:
# public model
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [8]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-finetuned-squadv2/checkpoint-24705/')
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-finetuned-squadv2/checkpoint-24705/', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
tokenizer.push_to_hub("hoannc0506/qa-squadv2-distilbert")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hoannc0506/qa-squadv2-distilbert/commit/5fc1011a6178fe684e142a52ed2d2542f26e3a80', commit_message='Upload tokenizer', commit_description='', oid='5fc1011a6178fe684e142a52ed2d2542f26e3a80', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
# init model
model = AutoModelForQuestionAnswering.from_pretrained('distilbert-finetuned-squadv2/checkpoint-24705/')

In [5]:
model

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [6]:
model.push_to_hub("hoannc0506/qa-squadv2-distilbert")

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hoannc0506/qa-squadv2-distilbert/commit/c68cca5133613136bf253a0c013e3d2ebaa24249', commit_message='Upload DistilBertForQuestionAnswering', commit_description='', oid='c68cca5133613136bf253a0c013e3d2ebaa24249', pr_url=None, pr_revision=None, pr_num=None)

## Inference with hugging face pipeline

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [3]:
model_name = "hoannc0506/qa-squadv2-distilbert"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device="cuda")

In [7]:
pipe.device

device(type='cuda')

In [8]:
input_question = "What's my name?"
input_context = "My name is Clara and I live in Berkeley."

In [9]:
pipe(question=input_question, context=input_context)

{'score': 0.9852074980735779, 'start': 11, 'end': 16, 'answer': 'Clara'}

## Evaluate on validation dataset

In [10]:
import evaluate
import numpy as np
from validation import compute_metrics
from datasets import load_dataset
from config import QAConfig
from data_utils import SquadDataProcessor
from transformers import Trainer

In [11]:
cfg = QAConfig(NUM_PROC=4)

In [12]:
data_processor = SquadDataProcessor(cfg)

In [21]:
# raw_val_dataset = load_dataset(cfg.DATASET_NAME, split ="validation[:10%]", num_proc=cfg.NUM_PROC)
raw_val_dataset = load_dataset(cfg.DATASET_NAME, split ="validation", num_proc=cfg.NUM_PROC)

In [22]:
val_data = data_processor.process_data(raw_val_dataset, data_type="validation")

Processing validation data


Tokenizing validation data (num_proc=4):   0%|          | 0/11873 [00:00<?, ? examples/s]

In [23]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [24]:
predictions, label_ids, metrics = trainer.predict(val_data)

In [25]:
start_logits, end_logits = predictions

In [26]:
start_logits[0].shape

(384,)

In [27]:
end_logits.shape

(12134, 384)

In [28]:
results = compute_metrics(
    start_logits,
    end_logits,
    val_data,
    raw_val_dataset
)
results

100%|██████████| 11873/11873 [00:19<00:00, 613.90it/s]


{'exact': 48.29444959151015,
 'f1': 52.42098419750788,
 'total': 11873,
 'HasAns_exact': 74.66261808367072,
 'HasAns_f1': 82.92752114996797,
 'HasAns_total': 5928,
 'NoAns_exact': 22.001682085786374,
 'NoAns_f1': 22.001682085786374,
 'NoAns_total': 5945,
 'best_exact': 65.71211993598922,
 'best_exact_thresh': -11.003822326660156,
 'best_f1': 67.46175087372762,
 'best_f1_thresh': -10.673299789428711}